In [ ]:
from sys import path
from os import getcwd

path.insert(getcwd() + '/modules/') # adding modules folder to path

from modules.DataMod import DataSet
from modules.TrainingManager import KerasTrainingManager
from modules.TrainingFunctions import generator_training
from modules.CustomLosses import LSSIM
from modules.misc import ssim_metric, psnrb_metric

from tensorflow.keras.optimizers import Adam


environ["CUDA_VISIBLE_DEVICES"] = "1"

## Fetching Datasets

In [ ]:
# creates the datasets
tinyDataSet = DataSet.load_rafael_tinyImagenet_64x64_noise_data()
cifarDataSet = DataSet.load_rafael_cifar_10_noise_data()
cifarAndTinyDataSet = DataSet.concatenateDataSets(cifarDataSet, tinyDataSet)

## Training Models

In [ ]:
# to do: 
# paralelize the training (does it's necessary?)
# batch size shoudn't be specified (keras API doc), does it affect the training?

In [ ]:
# uNet training

# trainiing unet only with tinyImagenet dataset
uNet_tiny_Manager = KerasTrainingManager(
    "Unet2.3-64x64.json",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},

    loss = LSSIM,
    
    fit_kwargs = {'batch_size' : 20, 'epochs' : 15, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric],

    training_function = generator_training,

    dataset = tinyDataSet
)

# trainiing unet only with cifar dataset
uNet_cifar_Manager = KerasTrainingManager(
    "Unet2.3-64x64.json",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},

    loss = LSSIM,
    
    fit_kwargs = {'batch_size' : 20, 'epochs' : 15, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric],

    training_function = generator_training,

    dataset = cifarDataSet
)

# training unet with both datasets concatenated
uNet_cifarAndTiny_Manager = KerasTrainingManager(
    "Unet2.3-64x64.json",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    loss = LSSIM,
    
    fit_kwargs = {'batch_size' : 20, 'epochs' : 15, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric],

    training_function = generator_training,

    dataset = cifarAndTinyDataSet
)

In [ ]:

# autoencoder training

# training autoencoder only with tinyImagenet dataset
autoencoder_tiny_Manager = KerasTrainingManager(
    "AutoEncoder-2.3-64x64",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    loss = LSSIM,

    fit_kwargs = {'batch_size' : 20, 'epochs' : 15, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric],

    training_function = generator_training,

    dataset = tinyDataSet
)

# training autoencoder only with cifar dataset
autoencoder_cifar_Manager = KerasTrainingManager(
    "AutoEncoder-2.3-64x64",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    loss = LSSIM

    fit_kwargs = {'batch_size' : 20, 'epochs' : 15, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric],

    training_function = generator_training,

    dataset = cifarDataSet
)

# training autoencoder with both datasets concatenated
autoencoder_cifarAndTiny_Manager = KerasTrainingManager(
    "AutoEncoder-2.3-64x64",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    loss = LSSIM,

    fit_kwargs = {'batch_size' : 20, 'epochs' : 15, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric],

    training_function = generator_training,

    dataset = cifarAndTinyDataSet
)


In [ ]:
# residual autoencoder training

# training residual autoencoder only with tinyImagenet dataset
residual_autoencoder_tiny_Manager = KerasTrainingManager(
    "ResidualAutoEncoder-0.1-64x64",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},

    loss = LSSIM,

    fit_kwargs = {'batch_size' : 20, 'epochs' : 25, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric],

    training_function = generator_training,

    dataset = tinyDataSet
)

# training residual autoencoder only with cifar dataset
residual_autoencoder_cifar_Manager = KerasTrainingManager(
    "ResidualAutoEncoder-0.1-64x64",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    loss = LSSIM,

    fit_kwargs = {'batch_size' : 20, 'epochs' : 25, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric],

    training_function = generator_training,

    dataset = cifarDataSet
)

# training residual autoencoder with both datasets concatenated
residual_autoencoder_cifarAndTiny_Manager = KerasTrainingManager(
    "ResidualAutoEncoder-0.1-64x64",
    optimizer = Adam,
    optimizer_kwargs = {'learning_rate' : 0.001, 'beta_1' : 0.9, 'beta_2' : 0.999, 'epsilon' : 1e-7, 'amsgrad' : False},
    
    loss = LSSIM,

    fit_kwargs = {'batch_size' : 20, 'epochs' : 25, 'verbose':1, 'validation_split':0, 'shuffle':True, 
    'class_weight':None, 'sample_weight':None, 'steps_per_epoch':None, 'validation_steps':None, 
    'validation_batch_size':None, 'validation_freq':1, 'max_queue_size':10, 'workers':1, 'use_multiprocessing':False},

    metrics = [ssim_metric],

    training_function = generator_training,

    dataset = cifarAndTinyDataSet
)

In [ ]:
# starting models training 
uNet_tiny_Manager.start_training()
uNet_cifar_Manager.start_training()
uNet_cifarAndTiny_Manager.start_training()

autoencoder_tiny_Manager.start_training()
autoencoder_cifar_Manager.start_training()
autoencoder_cifarAndTiny_Manager.start_training()

residual_autoencoder_tiny_Manager.start_training()
residual_autoencoder_cifar_Manager.start_training()
residual_autoencoder_cifarAndTiny_Manager.start_training()